# Grid of fixed aspect axes example

This example illustrates how to create a grid of axes with fixed-aspect ratios and margins around the figure.
The width of the figure is fixed (for example, the width could be constrained by the column width in a journal) while the height automatically adjusts from the given constraints and axes width.
This is difficult to accomplish directly in `matplotlib` without trial-and-error because the axes grid height isn't known until the figure is plotted.

In [ ]:
import itertools

import numpy as np

from mpllayout import solver
from mpllayout import primitives as pr
from mpllayout import constraints as co
from mpllayout import layout as lay
from mpllayout import matplotlibutils as lplt
from mpllayout import ui

## Specify the layout

In [ ]:
layout = lay.Layout()

## Create the figure box
layout.add_prim(pr.Quadrilateral(), "Figure")
layout.add_constraint(co.Box(), ("Figure",), ())

## Constrain the figure width
# Note that the figure height isn't directly constrainted because other
# constraints (margins, the axes aspect ratio, etc.) implicity determine what
# the figure height should be.
fig_width = 6
layout.add_constraint(co.Width(), ("Figure",), (fig_width,))

# Fix the figure bottom left corner to (0, 0)

layout.add_constraint(co.Fix(), ("Figure/Line0/Point0",), (np.array([0, 0]),))

## Create the axes
# You can change the number of rows and columns in the axes here.
num_row, num_col = (3, 4)
axes_shape = (num_row, num_col)
num_axes = int(np.prod(axes_shape))

axes_keys = [
    f"Axes[{row}, {col}]"
    for row, col in itertools.product(range(num_row), range(num_col))
]
for axes_key in axes_keys:
    layout.add_prim(pr.Axes(), axes_key)
    layout.add_constraint(co.Box(), (f"{axes_key}/Frame",), ())

## Constrain the axes in a grid

# Constrain them on a rectilinear grid
# (the x/y grid lines are still free to move left-right and up-down)
layout.add_constraint(
    co.RectilinearGrid(shape=axes_shape), [f"{axes_key}/Frame" for axes_key in axes_keys], ()
)

## Constrain the inter-axes margin in the grid and set a square aspect ratio

# First constrain the top-left corner axes aspect ratio to be square.
# Note this assumes the [0, 0] element is the top-left corner but other conventions
# are possible.
layout.add_constraint(co.AspectRatio(), ("Axes[0, 0]/Frame",), (1,))

# Because the axes lie on a grid, you only need to set widths/horizontal
# margins for a single row or axes, then heights/vertical margins for a single
# column of axes.
margin_inner = 0.1
for col in range(1, num_col):
    # Set equal widths for row 0
    layout.add_constraint(
        co.RelativeLength(),
        (f"Axes[0, {col}]/Frame/Line0", "Axes[0, 0]/Frame/Line0"),
        (1,)
    )
    # Set interior horizontal margin
    layout.add_constraint(
        co.OuterMargin(side='right'), (f"Axes[0, {col-1}]/Frame", f"Axes[0, {col}]/Frame"), (margin_inner,)
    )
for row in range(1, num_row):
    # Set equal heights for column 0
    layout.add_constraint(
        co.RelativeLength(),
        (f"Axes[{row}, 0]/Frame/Line1", "Axes[0, 0]/Frame/Line1"),
        (1,)
    )
    # Set interior vertical margin
    layout.add_constraint(
        co.OuterMargin(side='bottom'), (f"Axes[{row-1}, 0]/Frame", f"Axes[{row}, 0]/Frame"), (margin_inner,)
    )

# Note that the above can also be done in a single constraint with `co.Grid`

## Constrain margins around the figure

# Constrain top/bottom margins
margin_top = 0.2
margin_bottom = 0.2
layout.add_constraint(
    co.InnerMargin(side='top'), ("Axes[0, 0]/Frame", "Figure"), (margin_top,)
)
layout.add_constraint(
    co.InnerMargin(side='bottom'), (f"Axes[{num_row-1}, 0]/Frame", "Figure"), (margin_bottom, )
)

# Constrain left/right margins
margin_left = 0.2
margin_right = 0.2
layout.add_constraint(
    co.InnerMargin(side='left'),
    ("Axes[0, 0]/Frame", "Figure"), (margin_left,)
)
layout.add_constraint(
    co.InnerMargin(side='right'),
    (f"Axes[0, {num_col-1}]/Frame", "Figure"), (margin_right,)
)

## Solve the layout

In [ ]:
## Solve the constraints and form the figure/axes layout
prim_tree_n, info = solver.solve(layout)
print(info)

# This is the solved layout:
_fig, _ = ui.figure_prims(prim_tree_n)
_fig.savefig("grid_axes_layout.png", dpi=300)

## Plot a grid of images using the layout

In [ ]:
# Use the layout to plot randomly generated 10x10 pixel images
fig, axs = lplt.subplots(prim_tree_n)

for key, ax in axs.items():
    ax.set_axis_off()

    ax.imshow(np.random.rand(10, 10))

# x = np.linspace(0, 1)
# axs['Axes1'].plot(x, x**2)

fig.savefig("grid_axes.png")
# fig